<h1>Presentation du projet d'indexion de texte</h1>

In [1]:
from os import listdir
from os.path import isfile, join
from ipywidgets import FloatProgress
from IPython.display import display
from bs4 import BeautifulSoup as bs
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import nltk
import string
import operator
import shutil,os
import re

%load_ext autoreload
%autoreload 1
%aimport util_index
%aimport util_posting
%aimport fagin

In [2]:
WRITING_PATH_POSTING_LIST = "../data/saveFile/"
WRITING_PATH_UTIL = "../data/util/"
NAME_DOC_LIST = "docList"

<h1>Creation du index File</h1>

First we have to clean the repository of all old version of the algorithm.

In [3]:
util_index.cleanRepository(WRITING_PATH_POSTING_LIST)

FileNotFoundError: [Errno 2] No such file or directory: '../data/saveFile/'

In [ ]:
vocList = {}
docLenght = {}
util_index.buildIndexFile(vocList, docLenght, WRITING_PATH_POSTING_LIST = "../data/saveFile/")

writing in file, the format is :  "docId" "nombre Of word contained in the doc"

In [ ]:
util_index.writingDictInFile(docLenght, WRITING_PATH_UTIL, NAME_DOC_LIST, " ")

<h1> Creation Posting List</h1>

In [4]:
util_posting.createPostingList()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



[Errno 24] Too many open files: '/Users/jacques/Documents/INSA/OT1-INSA/src/../data/saveFile/postingList_27'
Traceback (most recent call last):
  File "/Users/jacques/Documents/INSA/OT1-INSA/src/util_posting.py", line 143, in createPostingList
    file_reader_last_read_list = initialize_file_readers()
  File "/Users/jacques/Documents/INSA/OT1-INSA/src/util_posting.py", line 65, in initialize_file_readers
    file_reader = open(os.path.join(savefile_path,file), "r")
OSError: [Errno 24] Too many open files: '/Users/jacques/Documents/INSA/OT1-INSA/src/../data/saveFile/postingList_27'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jacques/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-23523d84110c>", line 1, in <module>
    util_posting.createPostingList()
  File "/Users/jacques/Documents/INS

TypeError: must be str, not list

<h1>Fagins</h1>

In [ ]:
posting_lists = util_posting.creat_posting_list_obj_list("cat dog")

In [ ]:
test = fagin.top_k_thresh(posting_lists, 4, epsilon=0.0)

In [ ]:
print(test)